### data
TODO: choose probability/multi-hot/cnn embeddings to read<br />
TODO: choose good&bad/good&ugly to read<br />
TODO: make good&ugly data sizes same

In [1]:
import csv
import numpy as np
import pandas as pd

def read_csv(file_path, colEQ, colE, colQ):
    vecEQs = []
    vecEs = []
    vecQs = []
    labels = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            
            # str -> ndarray
            vecEQ = np.array(eval(row[colEQ]))
            
            vecE_str = row[colE]
            vecE_list = vecE_str.strip('[]').split()
            vecE = np.array([float(value) for value in vecE_list])
            
            vecQ_str = row[colQ]
            vecQ_list = vecQ_str.strip('[]').split()
            vecQ = np.array([float(value) for value in vecQ_list])
            
            label = float(row['label'])
            
            # append
            vecEQs.append(vecEQ)
            vecEs.append(vecE)
            vecQs.append(vecQ)
            labels.append(label)
    return np.array(vecEQs), np.array(vecEs), np.array(vecQs), np.array(labels)

def read_pickle(file_path):
    data = pd.read_pickle(file_path)
    #display(data)
    vecEQs = np.array([np.array((x)) for x in data['predEQ']])
    vecEs = np.array([np.array((x)) for x in data['embedE']])
    vecQs = np.array([np.array((x)) for x in data['embedQ']])
    labels = data['label'].astype(float).values
    return vecEQs, vecEs, vecQs, labels

In [2]:
import random
random.seed(42)

'''
# read prob.: good_embedding_probEQ.csv, bad_embedding_probEQ.csv

vecEQs_good, vecEs_good, vecQs_good, labels_good = read_csv('../embeddings/model_train/good_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')
#vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/model_train/bad_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/ugly_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')
'''
'''
# read multi-hot: good_embedding_predEQ.csv, bad_embedding_predEQ.csv

vecEQs_good, vecEs_good, vecQs_good, labels_good = read_csv('../embeddings/model_train/good_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
#vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/model_train/bad_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/ugly_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
'''


# read cnn: good_embedding.pkl, bad_embedding.pkl
vecEQs_good, vecEs_good, vecQs_good, labels_good = read_pickle('../embeddings/CNN(Resnet50)/good_embedding.pkl')
#vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_pickle('../embeddings/CNN(Resnet50)/bad_embedding.pkl')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_pickle('../embeddings/CNN(Resnet50)/ugly_embedding.pkl')
print("good data:", len(vecEQs_good), ", bad data:", len(vecEQs_bad))

#'''
# good 和 ugly 要一樣的資料量
sampling = np.random.choice(vecEQs_good.shape[0], vecEQs_bad.shape[0], replace=False)
vecEQs_good = vecEQs_good[sampling, :]
vecEs_good = vecEs_good[sampling, :]
vecQs_good = vecQs_good[sampling, :]
labels_good = labels_good[sampling]
#'''

# 合併 good & bad data, check shape
vecEQs = np.concatenate((vecEQs_good, vecEQs_bad), axis=0)
vecEs = np.concatenate((vecEs_good, vecEs_bad), axis=0)
vecQs = np.concatenate((vecQs_good, vecQs_bad), axis=0)
labels = np.concatenate((labels_good, labels_bad), axis=0)

print("vecEQs:", vecEQs.shape, "\nvecEs:", vecEs.shape, "\nvecQs:", vecQs.shape, "\nlabels:", labels.shape)

good data: 2094 , bad data: 1105
vecEQs: (2210, 4096) 
vecEs: (2210, 2048) 
vecQs: (2210, 2048) 
labels: (2210,)


In [9]:
# X: EQ, E0, 0Q
zeros = np.zeros_like(vecEs)
x1 = vecEQs
x2 = np.hstack((vecEs, zeros))
x3 = np.hstack((zeros, vecQs))
x123 = np.vstack((x1, x2, x3))

# X^: EQ, EQ, EQ
y123 = np.vstack((x1, x1, x1))

print("X:", x123.shape, "\nX^:", y123.shape)

X: (6630, 4096) 
X^: (6630, 4096)


### model
TODO: 注意 training 要丟入的 x, x^

In [10]:
# 196 >> 147 >> 98 >> 147 >> 196

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# given vecEQs
input_dim = vecEQs.shape[1]

# encoder layer (196 >> 147 >> 98)
encoding_dim1 = 147
encoding_dim2 = 98

input_layer = Input(shape=(input_dim,))
encoded1 = Dense(encoding_dim1, activation='relu')(input_layer)
encoded2 = Dense(encoding_dim2, activation='relu')(encoded1)

# decoder layer (98 >> 147 >> 196)
decoded1 = Dense(encoding_dim1, activation='relu')(encoded2)
decoded2 = Dense(input_dim, activation='sigmoid')(decoded1)

# encoder-decoder(196 >> 147 >> 98 >> 147 >> 196)
autoencoder = Model(input_layer, decoded2)

# encoder model
encoder = Model(input_layer, encoded2)

# decoder model
encoded_input = Input(shape=(encoding_dim2,))
decoder_layer1 = autoencoder.layers[-2]
decoder_layer2 = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer2(decoder_layer1(encoded_input)))

# compile model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4096)]            0         
                                                                 
 dense_8 (Dense)             (None, 147)               602259    
                                                                 
 dense_9 (Dense)             (None, 98)                14504     
                                                                 
 dense_10 (Dense)            (None, 147)               14553     
                                                                 
 dense_11 (Dense)            (None, 4096)              606208    
                                                                 
Total params: 1237524 (4.72 MB)
Trainable params: 1237524 (4.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# training
#autoencoder.fit(vecEQs, vecEQs, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)
autoencoder.fit(x123, y123, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

Epoch 1/50
21/21 [==============================] - 1s 34ms/step - loss: 0.1043 - val_loss: 0.0333
Epoch 2/50
21/21 [==============================] - 0s 22ms/step - loss: 0.0318 - val_loss: 0.0328
Epoch 3/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0311 - val_loss: 0.0328
Epoch 4/50
21/21 [==============================] - 0s 22ms/step - loss: 0.0308 - val_loss: 0.0326
Epoch 5/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0305 - val_loss: 0.0324
Epoch 6/50
21/21 [==============================] - 0s 21ms/step - loss: 0.0303 - val_loss: 0.0324
Epoch 7/50
21/21 [==============================] - 0s 22ms/step - loss: 0.0298 - val_loss: 0.0323
Epoch 8/50
21/21 [==============================] - 0s 23ms/step - loss: 0.0293 - val_loss: 0.0317
Epoch 9/50
21/21 [==============================] - 0s 23ms/step - loss: 0.0287 - val_loss: 0.0307
Epoch 10/50
21/21 [==============================] - 0s 22ms/step - loss: 0.0283 - val_loss: 0.0303
Epoch 11/

In [12]:
# middle layer
middle_vecEQs = encoder.predict(vecEQs)

70/70 [==============================] - 0s 2ms/step


### regression

In [13]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(middle_vecEQs, labels, test_size=0.2, random_state=42, stratify = labels)

# SVM model
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train, y_train)

# MSE, MAE
y_pred = svm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[SVM] MSE = {mse}, MAE = {mae}')

[SVM] MSE = 0.06797418337624203, MAE = 0.19813784304245097


In [14]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# MSE
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[RF] MSE = {mse}, MAE = {mae}')

[RF] MSE = 0.08457398190045248, MAE = 0.21418552036199096


In [15]:
from keras.models import Sequential
from keras.layers import Dense

# Neural Network model
nn_model = Sequential()
nn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(16, activation='sigmoid'))
nn_model.add(Dense(1))  # output layer

nn_model.compile(optimizer='adam', loss='mean_squared_error')

# training
nn_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.2)

# MSE
y_pred = nn_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[NN] MSE = {mse}, MAE = {mae}')

Epoch 1/20
45/45 [==============================] - 1s 5ms/step - loss: 0.2652 - val_loss: 0.1737
Epoch 2/20
45/45 [==============================] - 0s 3ms/step - loss: 0.1271 - val_loss: 0.1000
Epoch 3/20
45/45 [==============================] - 0s 4ms/step - loss: 0.0936 - val_loss: 0.0710
Epoch 4/20
45/45 [==============================] - 0s 3ms/step - loss: 0.0771 - val_loss: 0.0645
Epoch 5/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0717 - val_loss: 0.0610
Epoch 6/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0627 - val_loss: 0.0744
Epoch 7/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0654 - val_loss: 0.0559
Epoch 8/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0647 - val_loss: 0.0581
Epoch 9/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0591 - val_loss: 0.0577
Epoch 10/20
45/45 [==============================] - 0s 2ms/step - loss: 0.0570 - val_loss: 0.0585
Epoch 11/20
45/45 [

### 手動 record

In [9]:
import pandas as pd

data = {
    'probability': [0.2757061123247706, 0.2589729116945107, 0.24760135830766278],
    'multi-hot': [0.3113301803055666, 0.26886112194829226, 0.2524728533771422],
    'cnn': [0.25463821738103376, 0.23893257756563246, 0.24219229685222451]
}

df = pd.DataFrame(data, index=['svm', 'rf', 'nn'])
print("原始資料集(EQ)")
display(df)


data2 = {
    'probability': [0.2837730806873752, 0.2610291169451074, 0.2484090777202349],
    'multi-hot': [0.3188305040862045, 0.2613142604766219, 0.2565020791026134],
    'cnn': [0.23302369784904192, 0.2365852028639618, 0.24390136605141902]
}

print("\n三倍資料集(EQ, E0, 0Q)")
df2 = pd.DataFrame(data2, index=['svm', 'rf', 'nn'])
display(df2)

原始資料集(EQ)


,probability,multi-hot,cnn
svm,0.275706,0.311330,0.254638
rf,0.258973,0.268861,0.238933
nn,0.247601,0.252473,0.242192



三倍資料集(EQ, E0, 0Q)


,probability,multi-hot,cnn
svm,0.283773,0.318831,0.233024
rf,0.261029,0.261314,0.236585
nn,0.248409,0.256502,0.243901


In [16]:
# good & ugly

data3 = {
    'probability': [0.18668784933360047, 0.1716918552036199, 0.21571981347398259],
    'multi-hot': [0.16185558967853397, 0.16505469043740614, 0.1705426461395002],
    'cnn': [0.08310994265107106, 0.09175294117647059, 0.08578550574683298]
}

df3 = pd.DataFrame(data3, index=['svm', 'rf', 'nn'])
print("原始資料集(EQ)-good&ugly")
display(df3)


data4 = {
    'probability': [0.17874345017228893, 0.17046108597285067, 0.18968940847495594],
    'multi-hot': [0.17458910925342705, 0.1693084523924, 0.17449791095236505],
    'cnn': [0.06797418337624203, 0.08457398190045248, 0.05373021987323608]
}

print("\n三倍資料集(EQ, E0, 0Q)-good&ugly")
df4 = pd.DataFrame(data4, index=['svm', 'rf', 'nn'])
display(df4)

原始資料集(EQ)-good&ugly


,probability,multi-hot,cnn
svm,0.186688,0.161856,0.083110
rf,0.171692,0.165055,0.091753
nn,0.215720,0.170543,0.085786



三倍資料集(EQ, E0, 0Q)-good&ugly


,probability,multi-hot,cnn
svm,0.178743,0.174589,0.067974
rf,0.170461,0.169308,0.084574
nn,0.189689,0.174498,0.053730


### save model

In [17]:
import tensorflow as tf
import keras
import joblib

# CASE I: 訓練資料是 good=1 & bad=0
'''
# probability: 儲存原始資料集的 encoder + nn model
encoder.save('gb_prob_encoder_model.keras')
joblib.dump(nn_model, 'gb_prob_nn_model.pkl')
'''

'''
# multi-hot: 儲存原始資料集 encoder + nn model
encoder.save('gb_multihot_encoder_model.keras')
joblib.dump(nn_model, 'gb_multihot_nn_model.pkl')
'''

'''
# cnn: 儲存三倍資料集 encoder + svm model
encoder.save('gb_cnn_encoder_model.keras')
joblib.dump(svm_model, 'gb_cnn_svm_model.pkl')
'''

# CASE II: 訓練資料是 good=1 & ugly=0
'''
# probability: 儲存三倍資料集的 autoencoder, encoder + rf model
#autoencoder.save('autoencoder_model_prob3.keras')
encoder.save('gu_prob_encoder_model.keras')
joblib.dump(rf_model, 'gu_prob_rf_model.pkl')
'''

'''
# multi-hot: 儲存原始資料集 autoencoder, encoder + svm model
#autoencoder.save('autoencoder_model_multihot.keras')
encoder.save('gu_multihot_encoder_model.keras')
joblib.dump(svm_model, 'gu_multihot_svm_model.pkl')
'''


# cnn: 儲存三倍資料集 encoder + nn model
encoder.save('gu_cnn_encoder_model.keras')
joblib.dump(rf_model, 'gu_cnn_nn_model.pkl')


['gu_cnn_nn_model.pkl']

### end